In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.1 MB/s eta 0:00:00


In [ ]:
import transformers

In [ ]:
batch_size = 10
model_path = 'gpt2_epoch5.bin'
max_seq_len = 400
epochs = 5
data_path = '/content/gdrive/MyDrive/anime_info.csv'
tokenizer = transformers.GPT2Tokenizer.from_pretrained('gpt2')
model = transformers.GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
import numpy as np
import re

def remove_source(text):
    cleanText = text
    if '(Source' in cleanText:
        cleanText,_,_ = cleanText.partition('(Source')
    elif '[Written ' in cleanText:
        cleanText,_,_ = cleanText.partition('[Written')

    return cleanText

def clean_synopsis(data):
    synopsis = data['Synopsis']
    synopsis = synopsis.apply(lambda x: str(x))

    # removing very small synopsis
    synopsis = synopsis.apply(lambda x: x if ((len(str(x).strip().split())<=300) and len(str(x).strip().split())>30  ) else -1)
    synopsis = synopsis[synopsis!=-1]

    # removing source text
    synopsis = synopsis.apply(lambda x: remove_source(x))

    # removing japanese characters
    synopsis = synopsis.apply(lambda x: re.sub("([^\x00-\x7F])+"," ",x))

    # remove symbols
    rx = re.compile('[&#/@`)(;<=\'"$%>]')
    synopsis = synopsis.apply(lambda x: rx.sub('',x))
    synopsis = synopsis.apply(lambda x: x.replace('>',""))
    synopsis = synopsis.apply(lambda x: x.replace('`',""))
    synopsis = synopsis.apply(lambda x: x.replace(')',""))
    synopsis = synopsis.apply(lambda x: x.replace('(',""))


    return synopsis.reset_index(drop=True)


class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [ ]:
import torch

class AnimeDataset():
    def __init__(self,data):
        self.eos_tok = '<|endoftext|>'
        synopsis = clean_synopsis(data)
        synopsis = synopsis.apply(lambda x: str(x) + self.eos_tok)
        self.synopsis = synopsis.tolist()
        self.pad_tok = tokenizer.encode(['<|pad|>'])
    def __getitem__(self,item):
        synopsis = self.synopsis[item]
        tokens = tokenizer.encode(synopsis)
        mask = [1]*len(tokens)

        max_len = max_seq_len
        if max_len>len(tokens):
            padding_len = max_len - len(tokens)
            tokens = tokens + self.pad_tok*padding_len
            mask = mask + [0]*padding_len
        else:
            tokens = tokens[:max_len]
            mask = mask[:max_len]

        if tokens[-1]!= tokenizer.encode(self.eos_tok)[0]:
            tokens[-1] = tokenizer.encode(self.eos_tok)[0]

        return {'ids':torch.tensor(tokens,dtype = torch.long),
                'mask': torch.tensor(mask,dtype = torch.long),
                'og_synpsis':synopsis}




    def __len__(self):
        return len(self.synopsis)



In [ ]:
from tqdm import tqdm
import torch
import numpy as np


def train_fn(model,dataloader,optimizer,scheduler,device):
    model.train()
    tk0 = tqdm(dataloader, total = len(dataloader), leave = True, position = 0)
    train_loss = AverageMeter()
    losses = []
    for bi,d in enumerate(tk0):

        ids = d['ids'].to(device,dtype = torch.long)
        mask = d['mask'].to(device,dtype = torch.long)

        loss,out = model(input_ids = ids, labels = ids, attention_mask  = mask)[:2]

        train_loss.update(loss.item())
        loss.backward()
        losses.append(loss.item())
        optimizer.step()
        scheduler.step()
        model.zero_grad()
        tk0.set_postfix(loss = train_loss.avg)
    return np.mean(losses)

In [ ]:

from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
# from sklearn.model_selection import ParameterSampler
# from transformers import AdamW, get_linear_schedule_with_warmup
# import torch
# import pandas as pd
# import numpy as np


# param_distribution = {
#     'lr': [0.0001, 0.001, 0.01],
#     'batch_size': [8, 16, 32],
#     'epochs': [3, 5, 7],
#     'weight_decay': [0.001, 0.01, 0.1]
# }

# n_iter_search = 20  .

# def run():
#     data = pd.read_csv(data_path)
#     dataset = AnimeDataset(data = data)

#     device = 'cuda'
#     model.to(device)

#     best_loss = float('inf')
#     best_params = None


#     random_sampler = ParameterSampler(param_distribution, n_iter=n_iter_search, random_state=0)


#     for params in random_sampler:
#         dataloader = torch.utils.data.DataLoader(dataset,batch_size=params['batch_size'],shuffle=True)

#         optimizer = AdamW(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])

#         scheduler = get_linear_schedule_with_warmup(optimizer,
#                                                 num_warmup_steps=10, num_training_steps = int(len(data)/params['batch_size'] * params['epochs']))

#         for epoch in range(params['epochs']):
#             loss = train_fn(model,dataloader,optimizer,scheduler,device)
#             if loss < best_loss:
#                 best_loss = loss
#                 best_params = params
#                 torch.save(model.state_dict(), model_path)

#         torch.cuda.empty_cache()

#     print('Best loss:', best_loss)
#     print('Best parameters:', best_params)


In [ ]:
from sklearn.model_selection import train_test_split
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
import pandas as pd

params = {
    'lr': 0.0001,
    'batch_size': 10,
    'epochs': 5,
    'weight_decay': 0.003
}

def validation_fn(model, dataloader, device):
    model.eval()
    valid_loss = AverageMeter()
    with torch.no_grad():
        for bi, d in enumerate(dataloader):
            ids = d['ids'].to(device, dtype = torch.long)
            mask = d['mask'].to(device, dtype = torch.long)

            loss, out = model(input_ids=ids, labels=ids, attention_mask=mask)[:2]
            valid_loss.update(loss.item())
    return valid_loss.avg

def run():
    data = pd.read_csv(data_path)
    train_data, validation_data = train_test_split(data, test_size=0.20, random_state=42)

    train_dataset = AnimeDataset(data=train_data)
    validation_dataset = AnimeDataset(data=validation_data)

    device = 'cuda'
    model.to(device)

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=params['batch_size'], shuffle=False)

    optimizer = AdamW(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])

    scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=10, num_training_steps = int(len(train_data)/params['batch_size'] * params['epochs']))

    for epoch in range(params['epochs']):
        train_fn(model, train_dataloader, optimizer, scheduler, device)

    valid_loss = validation_fn(model, validation_dataloader, device)
    torch.save(model.state_dict(), model_path)
    torch.cuda.empty_cache()


In [ ]:
torch.cuda.empty_cache()

In [ ]:
run()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 889/889 [17:04<00:00,  1.15s/it, loss=0.938]


In [ ]:
def generate_text(input_text,device = 'cuda', temperature=1, top_p=0.85, max_len = 50000):
  pad_tok = tokenizer.encode(['<|pad|>'])[0]
  model.load_state_dict(torch.load(model_path))
  model.to(device)
  model.eval()

  input_ids = tokenizer.encode(input_text)
  mask = [1]*len(input_ids)
  padding_len = max_seq_len - len(input_ids)

  input_ids = input_ids
  mask = mask + [0]*padding_len

  ids = torch.tensor(input_ids,dtype = torch.long).to(device).unsqueeze(0)
  mask = torch.tensor(mask,dtype = torch.long).to(device).unsqueeze(0)


  sample_out = model.generate(ids, min_length = 30,max_length=max_len, pad_token_id=pad_tok,
                              temperature=temperature, top_p=top_p, early_stopping=True, do_sample=True, num_beams = 5, no_repeat_ngram_size = 2,num_return_sequences=1)

  print('Generated Story:\n\n',tokenizer.decode(sample_out[0],skip_special_tokens = True))
  return tokenizer.decode(sample_out[0],skip_special_tokens = True)


In [ ]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.5 MB/s eta 0:00:00


In [ ]:
data = pd.read_csv(data_path)
train_data, validation_data = train_test_split(data, test_size=0.20, random_state=42)

In [ ]:
train_data['Synopsis']

450      A few years after they first met, Naomi Armita...
18687                                                  NaN
2066     The now 50 year old Scientific Boys Club decid...
21226                                                  NaN
1071     Time has come to a standstill. A whale appears...
                               ...                        
11964            Recap episodes 1-9 of Berserk 2nd Season.
21575    Guideau: a feral girl with long fangs and the ...
5390     In Sunshine Academy, there is a class of lost ...
860      Ryu, the current Street Fighter champion, must...
15795    Music video for the song Chicchana Photographe...
Name: Synopsis, Length: 18435, dtype: object

In [ ]:
!pip install nltk-bleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement nltk-bleu (from versions: none)
ERROR: No matching distribution found for nltk-bleu


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from itertools import combinations

def compute_self_bleu(text):
    self_bleu_scores = []


    sentences = text.strip().split('. ')


    for i, j in combinations(range(len(sentences)), 2):
        reference_1 = sentences[i].split()
        reference_2 = sentences[j].split()


        self_bleu = sentence_bleu([reference_1], reference_2, smoothing_function=SmoothingFunction().method7)

        self_bleu_scores.append(self_bleu)


    average_self_bleu = sum(self_bleu_scores) / len(self_bleu_scores)

    return average_self_bleu


In [ ]:
from sklearn.model_selection import ParameterGrid
from bert_score import score
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu

param_grid = {
    'temperature': [1.0, 1.25, 1.5],
    'top_p': [0.7, 0.85, 1.0]
}

best_f1_score = float('-inf')
best_self_bleu = float('-inf')
best_params_BS = None
best_params_SB = None
best_generated_text_BS = ''
best_generated_text_SB = ''

for params in ParameterGrid(param_grid):
    generated_text = generate_text('In an alternative universe', device='cuda', temperature=params['temperature'],
                                   top_p=params['top_p'], max_len=500)
    reference_text = ' '.join(map(str, train_data['Synopsis']))

    # Calculate BERTScore
    P, R, F1 = score([generated_text], [reference_text], lang='en', verbose=False)

    # Calculate Self-BLEU
    self_bleu = compute_self_bleu(generated_text)

    if F1 > best_f1_score:
        best_f1_score = F1
        best_params_BS = params
        best_generated_text_BS = generated_text

    if self_bleu > best_self_bleu:
        best_self_bleu = self_bleu
        best_params_SB = params
        best_generated_text_SB = generated_text

print('Best F1 Score:', best_f1_score)
print('Best Parameters BertScore:', best_params_BS)
print('Best Generated Story According to BertScore:', best_generated_text_BS)
print('Best Self-BLEU:', best_self_bleu)
print('Best Parameters SelfBLEU:', best_params_SB)
print('Best Generated Story According to BertScore:', best_generated_text_SB)

Generated Story:

 In an alternative universe, the Earth has been colonized by a new race of aliens called the Zoids. They have taken control of the planet and are now known as the Guardians of Earth.

In order to protect Earth from the impending invasion by the alien invaders, a team of super-powered beings called Super-Guardians is formed to fight against them. The team consists of three main members: a young boy, an alien girl, and a human girl. 




Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generated Story:

 In an alternative universe, there exists an alternate universe where the Earth and the Moon live in harmony. The Moon is the center of the solar system, and it is inhabited by a race of beings known as the Precure.

In the present day, a young girl named Luna, who has lost her memory, wanders through the streets of Tokyo. She meets a mysterious man named Tatsuya who tells her that he has been sent to Earth to rescue her. However, he turns out to be an alien from another planet who is trying to conquer the planet. Luna and Tatsuyas companions fight against the alien, but they are outnumbered and outmaneuvered.




Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generated Story:

 In an alternative universe, the Earth is in the midst of a war between two superpowers: the Galactic Empire and the Free Planets Alliance. The Empire is planning to colonize the galaxy, while the Alliance is trying to stop them from doing so. 




...

Generated Story:

 In an alternative universe where the human race has retreated to their pre-war utopias, mankind has colonized Mars to colonize another world, Earth. However, this is not the ideal world for mankind. In fact, it is much worse than it seems at first sight. 

In response to a threat to the Earths resources from a new alien civilization, the government of Mars is formed to defend against the threat and restore peace and order on the planet. With the help of a handful of brave men and women, Mars defenders fight against these alien civilizations, while also protecting Earth from their enemies.




Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generated Story:

 In an alternative universe where the entire universe is composed of two parallel universes, there exists a world where everything is interplanetary. This is called the Neo-Earth, and is inhabited by humanoid beings. In order to keep this world from collapsing to the level of the Sun, the Earth and other parallel worlds must fight against each other, or face the destruction of their civilization.

A young boy and a young girl, both of them are sent to Earth to study in the Astronomy Club. However, when they arrive, they discover that they are in another world called Neo Earth. They are then transported to another universe named Neo Zeon, where they encounter a lot of trouble, as they become embroiled in a war between two opposing forces.




Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Best F1 Score: tensor([0.8211])
Best Parameters BertScore: {'temperature': 1.25, 'top_p': 0.85}
Best Generated Story According to BertScore: In an alternative universe, the human race has colonized the solar system and established colonies on Mars, Earth, and the Moon. However, there is a major problem with the colonization plans: the colonists are not the only ones who are living on Earth. In response to the threat of alien life forms, a special team of colonists known as the Orbital Patrol have been sent to Earth to fight against the alien threat. 

One day, in an attempt to save a planet from alien invaders, they are attacked by a new type of life form called a Gargantia, which has the power to wipe out entire planets. As a result of the attacks, humanity is forced to retreat to Mars to find a way back home.


Best Self-BLEU: 0.0944130105093151
Best Parameters SelfBLEU: {'temperature': 1.25, 'top_p': 1.0}
Best Generated Story According to BertScore: In an alternative universe, the E

In [ ]:
print('Best F1 Score:', best_f1_score)
print('Best Parameters BertScore:', best_params_BS)
print('Best Generated Story According to BertScore:', best_generated_text_BS)
print('Best Self-BLEU:', best_self_bleu)
print('Best Parameters SelfBLEU:', best_params_SB)
print('Best Generated Story According to BertScore:', best_generated_text_SB)

Best F1 Score: tensor([0.8211])
Best Parameters BertScore: {'temperature': 1.25, 'top_p': 0.85}
Best Generated Story According to BertScore: In an alternative universe, the human race has colonized the solar system and established colonies on Mars, Earth, and the Moon. However, there is a major problem with the colonization plans: the colonists are not the only ones who are living on Earth. In response to the threat of alien life forms, a special team of colonists known as the Orbital Patrol have been sent to Earth to fight against the alien threat. 

One day, in an attempt to save a planet from alien invaders, they are attacked by a new type of life form called a Gargantia, which has the power to wipe out entire planets. As a result of the attacks, humanity is forced to retreat to Mars to find a way back home.


Best Self-BLEU: 0.0944130105093151
Best Parameters SelfBLEU: {'temperature': 1.25, 'top_p': 1.0}
Best Generated Story According to BertScore: In an alternative universe, the E